In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from urllib.parse import quote

In [10]:
def scrape_jobs(position,location):
    position = quote(position, safe='-')
    location = quote(location, safe='-')
    page = 1
    data = []
    while True:
        template = 'https://www.jobstreet.co.id/id/{}-jobs/in-{}?page={}'
        url = template.format(position,location,page)
        
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'lxml')
        jobs = soup.find_all('div',class_='y735df0 _1akoxc50 _1akoxc54')
        print('jobs found', len(jobs),'url',url)
        zeroResult = soup.find('section',class_='y735df0 y735df1 _1iz8dgs82 _1iz8dgs6y _1iz8dgsaa _1iz8dgs96')
        if zeroResult:
            print('No more jobs found')
            break
        page += 1

        for job in jobs:
            try:
                job_role = job.find('h3').text.strip()
            except AttributeError:
                job_role = ''
            try:
                company_name = job.find('a',class_='y735df0 y735dff  y735df0 y735dff _842p0a0 _842p0a1').text.strip()
                # print(company_name)
            except AttributeError:
                company_name = ''
            try:
                location_company = job.find('a',class_='y735df0 _1iz8dgs4y _94v4w0 _94v4w1 _94v4w21 _1wzghjf4 _94v4w7').text.strip()
                # print(location_company)
            except AttributeError:
                location_company = ''
            try:
                posted_date = job.find('span',class_='y735df0 _1iz8dgs4y _94v4w0 _94v4w1 _94v4w22 _1wzghjf4 _94v4w7').span.text.strip()
                # print(posted_date)
            except AttributeError:
                posted_date = ''
            try:
                skill = (job.find('a',class_='y735df0 _1iz8dgsr _1iz8dgsf6 _1iz8dgsbu _1iz8dgs4y _1iz8dgsfm').text).split('subClassification:')[-1].strip()
                print(skill)
            except AttributeError:
                skill = ''
            
            # try:
            #     more_detail_link = job.header.h2.a['href']
            # except AttributeError:
            #     more_detail_link = ''
            
            extract_date = datetime.today().strftime('%Y-%m-%d')
            
            data.append({'Role':job_role, 'Company Name':company_name, 'Location':location_company, 'Posted Date':posted_date,'Extracted Date':extract_date,'Key Skill': skill})
    df = pd.DataFrame(data)
    
    df.to_csv(f'list_of_{position}_jobstreet.csv', index=False, encoding='utf-8')

        

In [11]:
if __name__ == '__main__':
    position = 'data analyst'
    location = 'jakarta'

    scrape_jobs(position,location)

jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=1
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=2
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=3
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=4
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=5
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=6
jobs found 104 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=7
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=8
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=9
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=10
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=11
jobs found 89 url https://www.

# Jobstreet

In [27]:
jobstreet_base_url ='https://www.jobstreet.co.id/id/jobs'

In [36]:
r = requests.get('https://www.jobstreet.co.id/id/jobs')
soup = BeautifulSoup(r.content, 'lxml')
jobs = soup.find_all('div',class_='y735df0 _1akoxc50 _1akoxc54')

In [58]:
for job in jobs:
    for job_card in job.find_all('div',class_='y735df0 _1iz8dgs6m'):
        if job_card:
            detailed_job_item = job_card.find('a', class_='y735df0 y735dff _1iz8dgs5i _1iz8dgsj _1iz8dgsk _1iz8dgsl _1iz8dgsm _1iz8dgs7', href=True)
            if detailed_job_item:
                detailed_url = detailed_job_item['href']
                print(detailed_url)